In [1]:
from Process import PlateReader
import pandas as pd
import matplotlib.pyplot as plt

### Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [3]:
sample_info = pd.read_csv(r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Temp or Working\06_22_21_CdSe_Maria.csv")
sample_info

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Temp or Working\\06_22_21_CdSe_Maria.csv'

### Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
         
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.

* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

In [22]:
path = r"C:\Users\Edwin\Downloads\06_16_21_ODE_SDS_2.xlsx"
plate_names = ['Sheet1']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

Should be length of sample_info + 1 (will remove the wavelength column shortly).

In [23]:
merged_df= PlateReader.merge_wavelength_dfs(plate_dfs)

***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [24]:
merged_dfi = PlateReader.detect_ovflw(merged_df, holder = 30)
merged_dfi

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
Wavelength,300.000,305.000,310.000,315.000,320.000,325.000,330.000,335.000,340.000,345.000,...,655.000,660.000,665.000,670.000,675.000,680.000,685.000,690.000,695.000,700.000
A1,0.266,0.184,0.153,0.135,0.118,0.104,0.094,0.083,0.074,0.068,...,0.039,0.039,0.039,0.039,0.040,0.040,0.040,0.040,0.040,0.040
A2,0.264,0.182,0.152,0.135,0.118,0.105,0.095,0.084,0.075,0.069,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.041
A3,0.261,0.180,0.150,0.133,0.116,0.103,0.093,0.082,0.073,0.068,...,0.038,0.038,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.038
A4,0.264,0.183,0.152,0.134,0.117,0.104,0.094,0.083,0.074,0.068,...,0.040,0.040,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H8,0.274,0.192,0.161,0.144,0.127,0.114,0.103,0.092,0.083,0.077,...,0.044,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043
H9,0.276,0.195,0.164,0.147,0.130,0.116,0.106,0.095,0.085,0.080,...,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045
H10,0.272,0.193,0.163,0.145,0.128,0.115,0.104,0.093,0.084,0.077,...,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.044,0.044,0.044
H11,0.268,0.189,0.160,0.143,0.126,0.113,0.103,0.091,0.082,0.076,...,0.044,0.044,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [25]:
platereader_df = PlateReader.rehead_wavelengths(merged_dfi)

platereader_df

,300.0nm,305.0nm,310.0nm,315.0nm,320.0nm,325.0nm,330.0nm,335.0nm,340.0nm,345.0nm,...,655.0nm,660.0nm,665.0nm,670.0nm,675.0nm,680.0nm,685.0nm,690.0nm,695.0nm,700.0nm
A1,0.266,0.184,0.153,0.135,0.118,0.104,0.094,0.083,0.074,0.068,...,0.039,0.039,0.039,0.039,0.040,0.040,0.040,0.040,0.040,0.040
A2,0.264,0.182,0.152,0.135,0.118,0.105,0.095,0.084,0.075,0.069,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.041
A3,0.261,0.180,0.150,0.133,0.116,0.103,0.093,0.082,0.073,0.068,...,0.038,0.038,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.038
A4,0.264,0.183,0.152,0.134,0.117,0.104,0.094,0.083,0.074,0.068,...,0.040,0.040,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039
A5,0.283,0.195,0.162,0.143,0.124,0.110,0.098,0.086,0.076,0.070,...,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H8,0.274,0.192,0.161,0.144,0.127,0.114,0.103,0.092,0.083,0.077,...,0.044,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043
H9,0.276,0.195,0.164,0.147,0.130,0.116,0.106,0.095,0.085,0.080,...,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.045
H10,0.272,0.193,0.163,0.145,0.128,0.115,0.104,0.093,0.084,0.077,...,0.045,0.045,0.045,0.045,0.045,0.045,0.045,0.044,0.044,0.044
H11,0.268,0.189,0.160,0.143,0.126,0.113,0.103,0.091,0.082,0.076,...,0.044,0.044,0.043,0.043,0.043,0.043,0.043,0.043,0.043,0.043


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [26]:
spectras = platereader_df
sample_info.reset_index(drop=True, inplace=True)
spectras.reset_index(drop=True, inplace=True)
combined_df = pd.concat([sample_info, spectras], axis = 1)[:len(sample_info)] # switch to the dicionary version since you could store a unblanked, blanked and a blanker in one file.
combined_df

,UID,Labware,Slot,Well,ODE concentration wtf,SDS concentration wtf,ethanol concentration wtf,water concentration wtf,ODE amount mass g,ODE amount volume uL,...,655.0nm,660.0nm,665.0nm,670.0nm,675.0nm,680.0nm,685.0nm,690.0nm,695.0nm,700.0nm
0,S2_A1_06_15_2021_ODE,Falcon 48 Well Plate 1500 ÂµL,2.0,A1,0.000,0.001,0.100000,0.899000,0.000,0.000000,...,0.039,0.039,0.039,0.039,0.040,0.040,0.040,0.040,0.040,0.040
1,S2_A2_06_15_2021_ODE,Falcon 48 Well Plate 1500 ÂµL,2.0,A2,0.000,0.001,0.177778,0.821222,0.000,0.000000,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.041
2,S2_A3_06_15_2021_ODE,Falcon 48 Well Plate 1500 ÂµL,2.0,A3,0.000,0.001,0.255556,0.743444,0.000,0.000000,...,0.038,0.038,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.038
3,S2_A4_06_15_2021_ODE,Falcon 48 Well Plate 1500 ÂµL,2.0,A4,0.000,0.001,0.333333,0.665667,0.000,0.000000,...,0.040,0.040,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039
4,S2_A5_06_15_2021_ODE,Falcon 48 Well Plate 1500 ÂµL,2.0,A5,0.000,0.001,0.411111,0.587889,0.000,0.000000,...,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039,0.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,S3_E2_06_15_2021_ODE,Falcon 48 Well Plate 1500 ÂµL,3.0,E2,0.001,0.001,0.488889,0.509111,0.001,1.267427,...,0.450,0.443,0.436,0.430,0.423,0.417,0.411,0.405,0.399,0.394
82,S3_E3_06_15_2021_ODE,Falcon 48 Well Plate 1500 ÂµL,3.0,E3,0.001,0.001,0.566667,0.431333,0.001,1.267427,...,0.474,0.469,0.463,0.458,0.452,0.447,0.441,0.436,0.431,0.426
83,S3_E4_06_15_2021_ODE,Falcon 48 Well Plate 1500 ÂµL,3.0,E4,0.001,0.001,0.644444,0.353556,0.001,1.267427,...,0.401,0.397,0.393,0.390,0.386,0.383,0.379,0.376,0.372,0.369
84,S3_E5_06_15_2021_ODE,Falcon 48 Well Plate 1500 ÂµL,3.0,E5,0.001,0.001,0.722222,0.275778,0.001,1.267427,...,0.311,0.310,0.309,0.308,0.306,0.306,0.304,0.303,0.303,0.302


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [27]:
temp_hold_path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Temp or Working\06_17_21_ODE_SDS_Unblanked_Abs"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)